In [91]:
# Import necessary modules/libraries.
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.stats import zscore
from ast import literal_eval

In [92]:
#Define dataframe from local CSV file
my_data = pd.read_csv('C:\\Users\\Joe\\Desktop\\tmdb_5000_movies.csv')

In [93]:
#Identify any potential duplicates by title; many remakes have the same title and should be retained.
my_data[my_data.duplicated(['title'],keep=False)]

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
972,44000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",NaN,72710,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,The Host,A parasitic alien soul is injected into the bo...,42.933027,"[{""name"": ""Nick Wechsler Productions"", ""id"": 8...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2013-03-22,63327201,125.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,You will be one of us,The Host,6.0,1817
1359,35000000,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na...",NaN,268,"[{""id"": 848, ""name"": ""double life""}, {""id"": 84...",en,Batman,The Dark Knight of Gotham City begins his war ...,44.104469,"[{""name"": ""PolyGram Filmed Entertainment"", ""id...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",1989-06-23,411348924,126.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Have you ever danced with the devil in the pal...,Batman,7.0,2096
2877,11000000,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 18, ""nam...",http://www.hostmovie.com/,1255,"[{""id"": 1261, ""name"": ""river""}, {""id"": 1880, ""...",ko,괴물,Gang-du is a dim-witted man working at his fat...,27.655270,"[{""name"": ""Cineclick Asia"", ""id"": 685}, {""name...","[{""iso_3166_1"": ""KR"", ""name"": ""South Korea""}]",2006-07-27,88489643,119.0,"[{""iso_639_1"": ""ko"", ""name"": ""\ud55c\uad6d\uc5...",Released,Monsters are real.,The Host,6.7,537
3647,0,"[{""id"": 18, ""name"": ""Drama""}]",NaN,39269,"[{""id"": 4470, ""name"": ""punk""}, {""id"": 10183, ""...",en,Out of the Blue,Dennis Hopper is a hard-drinking truck driver ...,0.679351,"[{""name"": ""Robson Street"", ""id"": 71953}]","[{""iso_3166_1"": ""CA"", ""name"": ""Canada""}]",1980-05-01,0,94.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A harrowing drama from the director of Easy Rider,Out of the Blue,6.5,17
3693,0,"[{""id"": 18, ""name"": ""Drama""}]",NaN,10844,"[{""id"": 2658, ""name"": ""new zealand""}, {""id"": 3...",en,Out of the Blue,Ordinary people find extraordinary courage in ...,0.706355,[],"[{""iso_3166_1"": ""NZ"", ""name"": ""New Zealand""}]",2006-10-12,0,103.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The true story of a small town massacre,Out of the Blue,5.9,18
4267,1377800,"[{""id"": 10751, ""name"": ""Family""}, {""id"": 12, ""...",NaN,2661,"[{""id"": 339, ""name"": ""submarine""}, {""id"": 849,...",en,Batman,The Dynamic Duo faces four super-villains who ...,9.815394,"[{""name"": ""Twentieth Century Fox Film Corporat...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1966-07-30,0,105.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,He's Here Big As Life In A Real Bat-Epic,Batman,6.1,203


In [94]:
# Generate new columns for cleaned attribute values; old values are retained for provenance.
my_data['genres_clean'] = 0
my_data['keywords_clean'] = 0
my_data['production_companies_clean'] = 0
my_data['roi'] = 0

In [102]:
#Must set new column as object type to allow insertion of lists into dataframes, otherwise Type error will occur.
my_data['genres_clean'] = my_data['genres_clean'].astype(object)
my_data['keywords_clean'] = my_data['keywords_clean'].astype(object)
my_data['production_companies_clean'] = my_data['production_companies_clean'].astype(object)
my_data['budget'] = my_data['budget'].astype(float)
my_data['revenue'] = my_data['revenue'].astype(float)
my_data['roi'] = my_data['roi'].astype(float)

In [103]:
# From genre attribute, this extracts values from 'name' keys as a list.
for i in range(len(my_data)):
    my_data.at[i, "genres_clean"] = [d['name'] for d in literal_eval(my_data.loc[i, "genres"])]
    

In [104]:
# From keywords attribute, this extracts values from 'name' keys as a list.
for i in range(len(my_data)):
    my_data.at[i, "keywords_clean"] = [d['name'] for d in literal_eval(my_data.loc[i, "keywords"])]
    

In [105]:
# From production_companies attribute, this extracts values from 'name' keys as a list.
for i in range(len(my_data)):
    my_data.at[i, "production_companies_clean"] = [d['name'] for d in literal_eval(my_data.loc[i, "production_companies"])]
    

In [106]:
# Mark to delete rows that contain value of 0 for budget.
for i in range(len(my_data)):
    if my_data.at[i, "budget"] == 0:
        my_data.at[i, "delete"] = 'Delete'

In [107]:
# Mark to delete rows that contain value of 0 for revenue.
for i in range(len(my_data)):
    if my_data.at[i, "revenue"] == 0:
        my_data.at[i, "delete"] = 'Delete'

In [108]:
# Remove homepage, language, overview attributes.
my_data.drop(columns=['original_language', 'homepage', 'overview'])

,budget,genres,id,keywords,original_title,popularity,production_companies,production_countries,release_date,revenue,...,status,tagline,title,vote_average,vote_count,genres_clean,keywords_clean,production_companies_clean,roi,delete
0,237000000.0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Avatar,150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2.787965e+09,...,Released,Enter the World of Pandora.,Avatar,7.2,11800,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Ingenious Film Partners, Twentieth Century Fo...",2.787965e+09,NaN
1,300000000.0,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",Pirates of the Caribbean: At World's End,139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,9.610000e+08,...,Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Walt Disney Pictures, Jerry Bruckheimer Films...",9.610000e+08,NaN
2,245000000.0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",Spectre,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,8.806746e+08,...,Released,A Plan No One Escapes,Spectre,6.3,4466,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Columbia Pictures, Danjaq, B24]",8.806746e+08,NaN
3,250000000.0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",The Dark Knight Rises,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1.084939e+09,...,Released,The Legend Ends,The Dark Knight Rises,7.6,9106,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Legendary Pictures, Warner Bros., DC Entertai...",1.084939e+09,NaN
4,260000000.0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",John Carter,43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,2.841391e+08,...,Released,"Lost in our world, found in another.",John Carter,6.1,2124,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...",[Walt Disney Pictures],2.841391e+08,NaN
5,258000000.0,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na...",559,"[{""id"": 851, ""name"": ""dual identity""}, {""id"": ...",Spider-Man 3,115.699814,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-01,8.908716e+08,...,Released,The battle within.,Spider-Man 3,5.9,3576,"[Fantasy, Action, Adventure]","[dual identity, amnesia, sandstorm, love of on...","[Columbia Pictures, Laura Ziskin Productions, ...",8.908716e+08,NaN
6,260000000.0,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...",38757,"[{""id"": 1562, ""name"": ""hostage""}, {""id"": 2343,...",Tangled,48.681969,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2010-11-24,5.917949e+08,...,Released,They're taking adventure to new lengths.,Tangled,7.4,3330,"[Animation, Family]","[hostage, magic, horse, fairy tale, musical, p...","[Walt Disney Pictures, Walt Disney Animation S...",5.917949e+08,NaN
7,280000000.0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",99861,"[{""id"": 882

In [109]:
# Delete all rows marked for deletion.
my_data.drop(my_data[my_data["delete"] == 'Delete'].index, inplace = True)

# Dataframe must be re-indexed after dropping rows, or this will causes error in subsequent loop functions.
my_data.reset_index(drop=True, inplace=True)

In [110]:
#Calculate return on investment a generate a new column.
for i in range(len(my_data)):
    my_data.at[i, "roi"] = round((my_data.at[i, "revenue"] - my_data.at[i, "budget"]) /  my_data.at[i, "budget"], 2)

In [111]:
#Generate separate a day of the year and year column.
for i in range(len(my_data)):
    d = datetime.strptime(my_data.at[i, "release_date"], '%Y-%m-%d')
    my_data.at[i, "day_of_year"] = d.timetuple().tm_yday
    my_data.at[i, "year"] = d.year


In [ ]:
#Create dummy variables; nominal/categorical variables must be converted for regression analysis.
genre_dummies = my_data["genres_clean"].str.join('|').str.get_dummies()

In [112]:
# Outlier analysis using z-score; adds columns to compute z-score for budget/revenue.
my_data['revenue_zscore'] = my_data[['revenue']].apply(zscore)
my_data['budget_zscore'] = my_data[['budget']].apply(zscore)
my_data['roi_zscore'] = my_data[['roi']].apply(zscore)

#Scatter plot with data standardized via z-score
#my_data.plot.scatter(x='budget_zscore', y='revenue_zscore', c='DarkBlue')

In [ ]:
""""
#Normalize data test
from sklearn import preprocessing
# Create x, where x the 'scores' column's values as floats
x = my_data[['budget_zscore']].values.astype(float)
y = my_data[['revenue_zscore']].values.astype(float)

# Create a minimum and maximum processor object
min_max_scaler = preprocessing.MinMaxScaler()

# Create an object to transform the data to fit minmax processor
x_scaled = min_max_scaler.fit_transform(x)
y_scaled = min_max_scaler.fit_transform(y)

# Run the normalizer on the dataframe
my_data['budget_normalized'] = pd.DataFrame(x_scaled)
my_data['revenue_normalized'] = pd.DataFrame(y_scaled)
my_data.plot.scatter(x='budget_normalized', y='revenue_normalized', c='DarkBlue')

"""

In [113]:
# Analyze Spearman correlation coefficient for numerical attributes.
corr = my_data.corr(method ='spearman')
corr.style.background_gradient(cmap='coolwarm')

,budget,id,popularity,revenue,runtime,vote_average,vote_count,roi,day_of_year,year,revenue_zscore,budget_zscore,roi_zscore
budget,1,-0.0455988,0.461416,0.678594,0.242245,-0.13709,0.483212,-0.13088,0.0222923,0.2278,0.678594,1,-0.13088
id,-0.0455988,1,-0.0769209,-0.155971,-0.150518,-0.23064,-0.0995209,-0.175681,-0.0640948,0.679675,-0.155971,-0.0455988,-0.175681
popularity,0.461416,-0.0769209,1,0.70344,0.202135,0.335036,0.939383,0.432185,0.0243514,0.273475,0.70344,0.461416,0.432185
revenue,0.678594,-0.155971,0.70344,1,0.2289,0.125872,0.747607,0.573088,0.0415268,0.13784,1,0.678594,0.573088
runtime,0.242245,-0.150518,0.202135,0.2289,1,0.387016,0.204455,0.041287,0.140079,-0.0698464,0.2289,0.242245,0.041287
vote_average,-0.13709,-0.23064,0.335036,0.125872,0.387016,1,0.360959,0.334632,0.136344,-0.146052,0.125872,-0.13709,0.334632
vote_count,0.483212,-0.0995209,0.939383,0.747607,0.204455,0.360959,1,0.467844,0.0166128,0.277291,0.747607,0.483212,0.467844
roi,-0.13088,-0.175681,0.432185,0.573088,0.041287,0.334632,0.467844,1,0.0244166,-0.0964575,0.573088,-0.13088,1
day_of_year,0.0222923,-0.0640948,0.0243514,0.0415268,0.140079,0.136344,0.0166128,0.0244166,1,-0.0826832,0.0415268,0.0222923,0.0244166
year,0.2278,0.679675,0.273475,0.13784,-0.0698464,-0.146052,0.277291,-0.0964575,-0.0826832,1,0.13784,0.2278,-0.0964575


In [114]:
# Analyze Spearman correlation coefficient for genres; 
# this aids in determining if genres can be combined or removed and still retain variance.
corr = genre_dummies.corr(method ='spearman')
corr.style.background_gradient(cmap='coolwarm')

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,Thriller,War,Western
Action,1,0.362567,-0.092205,-0.185851,0.143489,-0.0560453,-0.238484,-0.123091,0.0664183,-0.0248204,-0.00074024,-0.0845134,-0.0887707,-0.0608821,-0.219424,0.253288,0.290907,0.046763,0.0406404
Adventure,0.362567,1,0.185245,-0.0714681,-0.130727,-0.0553645,-0.256282,0.233389,0.241914,-0.0199798,-0.0321793,-0.131317,-0.0746613,-0.0650104,-0.143549,0.234198,-0.012525,0.00582313,0.0310763
Animation,-0.092205,0.185245,1,0.106852,-0.10906,-0.0271331,-0.185966,0.596239,0.125014,0.0238438,-0.0539135,-0.0841716,0.0329367,-0.0695269,-0.0913944,-0.0159202,-0.14999,-0.0418568,-0.0132439
Comedy,-0.185851,-0.0714681,0.106852,1,-0.090575,-0.0487501,-0.230012,0.188478,0.0157521,-0.0119197,-0.138045,-0.161288,0.0280676,-0.17365,0.188769,-0.140267,-0.367185,-0.118063,-0.0475062
Crime,0.143489,-0.130727,-0.10906,-0.090575,1,-0.0478655,0.0685808,-0.14861,-0.126345,-0.0172736,-0.058522,-0.0791892,-0.0550342,0.129571,-0.13348,-0.132543,0.302814,-0.0772724,-0.00765248
Documentary,-0.0560453,-0.0553645,-0.0271331,-0.0487501,-0.0478655,1,-0.0864124,-0.011749,-0.0375593,0.0687374,0.0179378,-0.0369422,0.0897447,-0.0326296,-0.0507402,-0.0428295,-0.0696687,-0.0214392,-0.0146285
Drama,-0.238484,-0.256282,-0.185966,-0.230012,0.0685808,-0.0864124,1,-0.200467,-0.161198,0.0121788,0.202427,-0.193144,0.0494593,0.0334546,0.203435,-0.218616,-0.038818,0.143093,-0.0115307
Family,-0.123091,0.233389,0.596239,0.188478,-0.14861,-0.011749,-0.200467,1,0.271242,0.03569,-0.0774082,-0.117632,0.0614817,-0.0960537,-0.0764511,-0.00782075,-0.219287,-0.0701359,-0.0255735
Fantasy,0.0664183,0.241914,0.125014,0.0157521,-0.126345,-0.0375593,-0.161198,0.271242,1,-0.0135543,-0.0746305,0.0127112,-0.0152266,-0.0442474,-0.0257846,0.0543024,-0.108792,-0.0516585,-0.0308535
Foreign,-0.0248204,-0.0199798,0.0238438,-0.0119197,-0.0172736,0.0687374,0.0121788,0.03569,-0.0135543,1,-0.00853914,-0.0133316,-0.00743038,-0.0117753,0.0228919,-0.0154562,-0.0251418,-0.00773691,-0.00527908


In [ ]:
# Show first 25 rows of dataframe
my_data.head(25)

In [ ]:
#Save dataframe as CSV somewhere
#my_data.to_csv('C:\\Users\\Joe\\Desktop\\my_data_scores.csv')

In [ ]:
# Strong correlation between budget and revenue is apparent;few outliers with small budgets, but high revenue.
my_data.plot.scatter(x='budget', y='revenue', loglog=True, legend=False)
plt.show()

In [ ]:
my_data.plot.scatter(x='revenue', y='roi', legend=False)
plt.show()

In [ ]:
my_data